# Feature Engineering



- `Moving Average`: This indicator reveals price trends by smoothing out market fluctuations, effectively filtering out short-term noise from price data.
- `Momentum (MOM)`: This metric quantifies the acceleration of a security's price or volume, essentially measuring how rapidly prices are changing in a given direction.
- `Rate Of Change (ROC)`: This momentum oscillator calculates the percentage difference between current price and the price n periods ago. Assets displaying **elevated ROC values** typically signal overbought conditions, while **depressed ROC readings** often indicate oversold market states.
- `Relative Strength Index (RSI)`: A momentum measurement that evaluates the speed and magnitude of recent price movements to identify potential overbought or oversold conditions. It operates on a scale of 0-100, with **readings above 70 suggesting an overbought asset** and **readings below 30 indicating an undervalued, oversold condition**.
- `Stochastic Oscillator (%K and %D)`: This momentum tool compares a security's closing price to its price range over a specific timeframe. The %K line represents the faster signal, while %D serves as the slower, more smoothed indicator.

In [1]:
import os
import pandas as pd

pd.set_option("display.precision", 2)

PROCESSED_PATH = "data/processed/"
TRAIN_DATA_NAME = "train_data.csv"  # 60%
VALIDATION_DATA_NAME = "validation_data.csv"  # 20%
TEST_DATA_NAME = "test_data.csv"  # 20%

TRAIN_DATA_FILE = os.path.join(PROCESSED_PATH, TRAIN_DATA_NAME)
VALIDATION_DATA_FILE = os.path.join(PROCESSED_PATH, VALIDATION_DATA_NAME)

train_data = pd.read_csv(TRAIN_DATA_FILE)
valid_data = pd.read_csv(VALIDATION_DATA_FILE)

train_data.set_index("Timestamp", inplace=True)
valid_data.set_index("Timestamp", inplace=True)